In [ ]:
import pandas as pd
import openai

In [ ]:
df_rando = pd.read_csv("../data/jacques_cartiers/Sheet 2-Table 1.csv")

In [ ]:
question = "Je souhaite faire une randonnée au parc de la jacques-cartier, nous avons un chien avec nous et un enfant qui sera dans un sac de transport. Quelle randonnée vous nous recommendez? "

In [ ]:
df_rando

In [ ]:
doc_prompt_cfg = {
        "max_tokens": 3500,
        "text_before_documents": (
            "Vous etes un assistant qui soumet des recommendations pour les activités ou les hébergements de la Sépaq, un organisme mettant de l'avant les lieux de nature au Québec. "
            "Voici la liste de randonnées"
            "<DOCUMENTS> "
        ),
        "text_before_prompt": (
            "<\DOCUMENTS>\n"
            "Réponds à la question suivante:\n"  
        )
      }

In [ ]:
def format_doc(matched_documents, max_tokens):
        """Format our matched documents to plaintext.
        We also make sure they fit in the alloted max_tokens space.
        """
        documents_str = ""
        total_tokens = 0

        num_total_docs = len(matched_documents)
        num_preserved_docs = 0
        for _, row in matched_documents.iterrows():
            doc = f"""
            Randonnée pédestre seulement: {row['Randonnée pédestre seulement']}\n
            Description: {row['Description']}\n
            Longueur et durée: {row['Longueur et durée']}\n
            Degré de difficulté: {row['Degré de difficulté']}\n
            Lieu de départ: {row['Lieu de départ']}
            """
            num_preserved_docs += 1
            token_count = len(doc.split())
            if total_tokens + token_count <= max_tokens:
                documents_str += f"<DOCUMENT>{doc}<\\DOCUMENT>"
                total_tokens += token_count
            else:
                remaining_tokens = max_tokens - total_tokens
                truncated_doc = " ".join(doc.split()[:remaining_tokens])
                documents_str += f"<DOCUMENT>{truncated_doc}<\\DOCUMENT>"
                break

        if num_preserved_docs < (num_total_docs):
            matched_documents = matched_documents.iloc[:num_preserved_docs]

        return documents_str

In [ ]:
docs = format_doc(df_rando, max_tokens=3000)

In [ ]:
def format_prompt(documents_str, config):
        """
        Prepare the system prompt with prompt engineering.

        Joins the text before and after documents with
        """
        formatter = "{text_before_documents}\n{documents}\n{text_before_prompt}"
        system_prompt = formatter.format(
            text_before_documents=config["text_before_documents"], documents=documents_str, text_before_prompt=config["text_before_prompt"]
        )
        token_count = len(system_prompt.split())
        if token_count > config["max_tokens"]:
            raise ValueError(f"System prompt tokens > {config['max_tokens']}")
        return system_prompt

In [ ]:
simple_prompt_cfg = {
            "max_tokens": 3500,
            "text_before_documents": "You are a chatbot answering questions.\n",
            "text_before_prompt": "Answer the following question:\n",
        }

In [ ]:
prompt = format_prompt(docs, doc_prompt_cfg)

In [ ]:
prompt_w_question = prompt + question

In [ ]:
response = openai.Completion.create(
          model="text-davinci-003",
          prompt=prompt_w_question,
          temperature=0.8,
          max_tokens=1000,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
      )

In [ ]:
response

In [ ]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": f"{prompt_w_question}"},
    {"role": "assistant", "content": f"Je vous recommande la randonn\u00e9e p\u00e9destre \"La Tourbi\u00e8re\" au parc de la Jacques-Cartier. Cette randonn\u00e9e est facile et dure environ 1 heure. Les chiens sont admis dans ce sentier, selon certaines conditions, ce qui est id\u00e9al pour vous. De plus, la randonn\u00e9e est relativement courte, ce qui convient \u00e0 votre enfant qui sera dans un sac de transport. Vous pourrez d\u00e9couvrir une tourbi\u00e8re, une sapini\u00e8re et une for\u00eat de bouleaux jaunes lors de cette randonn\u00e9e. Le lieu de d\u00e9part se trouve au Chemin du Parc-National au km 3."},
    {"role": "user", "content": f"Merci pour la suggestion, mais je cherche une randonnée un peu plus longue, toute en étant acceptée pour les chiens. Une idée?"}
  ],
  temperature=0.5
)

print(completion.choices[0].message)

In [ ]:
print(completion.choices[0].message)

In [ ]:
response